In [19]:
import pandas as pd

In [20]:
census_data = pd.read_csv(r"C:/Users/krish/OneDrive/Documentos/acs2017_census_tract_data.csv")
print(census_data.head())

      TractId    State          County  TotalPop   Men  Women  Hispanic  \
0  1001020100  Alabama  Autauga County      1845   899    946       2.4   
1  1001020200  Alabama  Autauga County      2172  1167   1005       1.1   
2  1001020300  Alabama  Autauga County      3385  1533   1852       8.0   
3  1001020400  Alabama  Autauga County      4267  2001   2266       9.6   
4  1001020500  Alabama  Autauga County      9965  5054   4911       0.9   

   White  Black  Native  ...  Walk  OtherTransp  WorkAtHome  MeanCommute  \
0   86.3    5.2     0.0  ...   0.5          0.0         2.1         24.5   
1   41.6   54.5     0.0  ...   0.0          0.5         0.0         22.2   
2   61.4   26.5     0.6  ...   1.0          0.8         1.5         23.1   
3   80.3    7.1     0.5  ...   1.5          2.9         2.1         25.9   
4   77.5   16.4     0.0  ...   0.8          0.3         0.7         21.0   

   Employed  PrivateWork  PublicWork  SelfEmployed  FamilyWork  Unemployment  
0       881  

In [21]:
required_columns = ['County', 'State', 'TotalPop', 'IncomePerCap', 'Poverty']
filtered_data = census_data[required_columns].copy()

# Create a new column 'ID' as a unique identifier for the county
filtered_data['ID'] = range(len(filtered_data))

# Group the data by 'County' and 'State' and aggregate the values using appropriate aggregation functions
aggregated_data = filtered_data.groupby(['County', 'State']).agg({'TotalPop': 'sum', 'IncomePerCap': 'mean', 'Poverty': 'mean'})

In [22]:
# Reset the index of the aggregated DataFrame
County_Info = aggregated_data.reset_index()

# Rename the columns to match the desired column names
County_Info.rename(columns={'County': 'county', 'TotalPop': 'Population', 'IncomePerCap': 'PerCapitaIncome'}, inplace=True)

# Reset the 'ID' column to start from 1
County_Info['ID'] = County_Info.index + 1

County_Info

# Define input counties
county_inputs = ["Loudoun County, Virginia", "Washington County, Oregon", "Harlan County, Kentucky", "Malheur County, Oregon"]

county_details = pd.DataFrame(columns=County_Info.columns)

for input_county in county_inputs:
    county_name, state_name = input_county.split(", ")
    details = County_Info[(County_Info['county'] == county_name) & (County_Info['State'] == state_name)]
    county_details = pd.concat([county_details,details])

print("County details:\n", county_details)

County details:
                  county     State Population  PerCapitaIncome    Poverty    ID
1759     Loudoun County  Virginia     374558     50391.015625   3.884375  1760
3046  Washington County    Oregon     572071     34970.817308  10.446154  3047
1234      Harlan County  Kentucky      27548     16010.363636  33.318182  1235
1822     Malheur County    Oregon      30421     17966.428571  24.414286  1823


C:\Users\krish\AppData\Local\Temp\ipykernel_408\1132893232.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  county_details = pd.concat([county_details,details])


In [23]:
covid_data = pd.read_csv(r"C:/Users/krish/OneDrive/Documentos/COVID_county_data.csv")
covid_data

# Create a unique ID for each county-state combination
covid_data['ID'] = covid_data['county'] + '_' + covid_data['state']
covid_data['ID'] = pd.factorize(covid_data['ID'])[0] + 1

covid_data

# Convert date column to datetime
covid_data['date'] = pd.to_datetime(covid_data['date'], format='%Y-%m-%d')
covid_data['month'] = covid_data['date'].dt.to_period('M')
covid_data

,date,county,state,fips,cases,deaths,ID,month
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0,1,2020-01
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0,1,2020-01
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0,1,2020-01
3,2020-01-24,Cook,Illinois,17031.0,1,0.0,2,2020-01
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0,1,2020-01
...,...,...,...,...,...,...,...,...
1050253,2021-02-20,Sweetwater,Wyoming,56037.0,3645,34.0,1357,2021-02
1050254,2021-02-20,Teton,Wyoming,56039.0,3318,9.0,640,2021-02
1050255,2021-02-20,Uinta,Wyoming,56041.0,2024,12.0,2305,2021-02
1050256,2021-02-20,Washakie,Wyoming,56043.0,876,26.0,1784,2021-02


In [24]:
covid_data_monthly = covid_data.groupby(['ID', 'month']).agg({
    'cases': 'sum',
    'deaths': 'sum',
    'county': 'first'
}).reset_index()

covid_data_monthly['ID'] = range(1, len(covid_data_monthly) + 1)
covid_data_monthly = covid_data_monthly[['ID', 'month', 'cases', 'deaths', 'county']].copy()

covid_data_monthly.columns = ['ID', 'Month', 'cases', 'deaths', 'county']
selected_months = ['2020-08', '2021-01', '2021-02']
selected_county = 'Malheur'
covid_report = covid_data_monthly[(covid_data_monthly['county'] == selected_county) & 
                              (covid_data_monthly['Month'].astype(str).isin(selected_months))]

print(covid_report)



          ID    Month  cases  deaths   county
25593  25594  2020-08  28163   459.0  Malheur
25598  25599  2021-01  96297  1627.0  Malheur
25599  25600  2021-02  65951  1137.0  Malheur


In [25]:
covid_summary = (covid_data_monthly.groupby("ID").agg({"cases": "sum", "deaths": "sum"}).reset_index())

covid_summary.rename(columns={"cases": "TotalCases", "deaths": "TotalDeaths"}, inplace=True)
covid_summary = pd.merge(covid_summary, County_Info, how="left", on="ID")
covid_summary["TotalCasesPer100K"] = round((covid_summary["TotalCases"]) / (covid_summary["Population"] / 100000), 2)
covid_summary["TotalDeathsPer100K"] = round((covid_summary["TotalDeaths"]) / (covid_summary["Population"] / 100000), 2)

counties = [('Washington County', 'Oregon'), ('Malheur County', 'Oregon'), ('Loudoun County', 'Virginia'), ('Harlan County', 'Kentucky')]

# Filter the DataFrame based on the specified counties
filtered_df = covid_summary[
    (covid_summary['county'].isin([county[0] for county in counties])) &
    (covid_summary['State'].isin([county[1] for county in counties]))
]

# Extract the 'Poverty%' and 'TotalCasesPer100K' values for the specified counties
poverty_percentage = filtered_df[['county', 'State', 'Poverty']]
total_cases_per_100k = filtered_df[['county', 'State', 'TotalCasesPer100K']]

# Print the results
print("poverty%:")
print(poverty_percentage)
print("\nTotal Cases/100K:")
print(total_cases_per_100k)

poverty%:
                 county     State    Poverty
1234      Harlan County  Kentucky  33.318182
1759     Loudoun County  Virginia   3.884375
1822     Malheur County    Oregon  24.414286
3035  Washington County  Kentucky  16.900000
3046  Washington County    Oregon  10.446154
3053  Washington County  Virginia  14.715385

Total Cases/100K:
                 county     State  TotalCasesPer100K
1234      Harlan County  Kentucky         1671979.82
1759     Loudoun County  Virginia            3086.84
1822     Malheur County    Oregon          542792.81
3035  Washington County  Kentucky          178864.91
3046  Washington County    Oregon           61741.46
3053  Washington County  Virginia              55.09
